In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/covid19-global-forecasting-week-1/submission.csv
/kaggle/input/covid19-global-forecasting-week-1/test.csv
/kaggle/input/covid19-global-forecasting-week-1/train.csv


In [2]:
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor
import matplotlib.pyplot as plt
from sklearn import metrics

In [3]:
df_train = pd.read_csv('../input/covid19-global-forecasting-week-1/train.csv', infer_datetime_format=True)
df_test = pd.read_csv('../input/covid19-global-forecasting-week-1/test.csv', infer_datetime_format=True)

In [4]:
datetime_array = np.array([df_train['Date'][0]])
datetime_array1 = np.array([df_test['Date'][0]])
df_train = df_train.drop(['Province/State','Country/Region'], axis=1)
df_test = df_test.drop(['Province/State', 'Country/Region'], axis=1)
df_train = df_train.dropna()

df_test["Lat"]  = df_test["Lat"].fillna(12.5211)
df_test["Long"]  = df_test["Long"].fillna(69.9683)

In [5]:
inferred_datetime_format = \
        pd.core.tools.datetimes._guess_datetime_format_for_array(
            datetime_array)
inferred_datetime_format1 = \
        pd.core.tools.datetimes._guess_datetime_format_for_array(
            datetime_array1)

In [6]:
df_master_data_dt = pd.to_datetime(df_train['Date'],
                                       format=inferred_datetime_format)
df_test_data_dt = pd.to_datetime(df_test['Date'],
                                       format=inferred_datetime_format1)

In [7]:
df_train['Year'] = df_master_data_dt.dt.year
df_train['Origin Day of Week'] = \
df_master_data_dt.dt.dayofweek
df_train['Origin Month'] = df_master_data_dt.dt.month
df_train['Origin Week'] = df_master_data_dt.dt.week

df_test['Year'] = df_test_data_dt.dt.year
df_test['Origin Day of Week'] = \
df_test_data_dt.dt.dayofweek
df_test['Origin Month'] = df_test_data_dt.dt.month
df_test['Origin Week'] = df_test_data_dt.dt.week

In [8]:
df_train["Date"] = df_train["Date"].apply(lambda x: x.replace("-",""))
df_train["Date"]  = df_train["Date"].astype(int)

df_test["Date"] = df_test["Date"].apply(lambda x: x.replace("-",""))
df_test["Date"]  = df_test["Date"].astype(int)

In [9]:
features = ["Lat","Long","Date","Year","Origin Day of Week","Origin Month","Origin Week"]

x = df_train[features]
y = df_train['ConfirmedCases']
y1 = df_train['Fatalities']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=23, shuffle=False)

x_train1, x_test1, y_train1, y_test1 = train_test_split(x, y1, test_size=0.2, random_state=23, shuffle=False)

In [10]:
cb_model = CatBoostRegressor(iterations=5000, learning_rate=0.0001, max_depth=15)
model = cb_model.fit(x_train, y_train, use_best_model=True, verbose=True)

predictions = model.predict(x_test)
pred1 = model.predict(df_test)
pred1 = pd.DataFrame(pred1)
pred1.columns = ["ConfirmedCases_prediction"]

You should provide test set for use best model. use_best_model parameter has been switched to false value.


0:	learn: 3936.7916124	total: 76.8ms	remaining: 6m 24s
1:	learn: 3936.5725926	total: 158ms	remaining: 6m 36s
2:	learn: 3936.3013523	total: 191ms	remaining: 5m 18s
3:	learn: 3936.0252843	total: 195ms	remaining: 4m 3s
4:	learn: 3935.8273441	total: 542ms	remaining: 9m 1s
5:	learn: 3935.6670514	total: 728ms	remaining: 10m 6s
6:	learn: 3935.4191472	total: 739ms	remaining: 8m 46s
7:	learn: 3935.1428441	total: 756ms	remaining: 7m 51s
8:	learn: 3934.8641875	total: 782ms	remaining: 7m 13s
9:	learn: 3934.6401074	total: 790ms	remaining: 6m 34s
10:	learn: 3934.3786825	total: 901ms	remaining: 6m 48s
11:	learn: 3934.1525711	total: 916ms	remaining: 6m 20s
12:	learn: 3933.8717633	total: 959ms	remaining: 6m 8s
13:	learn: 3933.5971819	total: 963ms	remaining: 5m 42s
14:	learn: 3933.3086088	total: 1.15s	remaining: 6m 22s
15:	learn: 3933.0439043	total: 1.17s	remaining: 6m 3s
16:	learn: 3932.7921358	total: 1.18s	remaining: 5m 44s
17:	learn: 3932.5063204	total: 1.19s	remaining: 5m 28s
18:	learn: 3932.2576236

In [11]:
cb_model1 = CatBoostRegressor(iterations=2000, learning_rate=0.0001, max_depth=15)
model1 = cb_model1.fit(x_train1, y_train1, use_best_model=True, verbose=True)
predictions1 = model1.predict(x_test1)

pred2 = model1.predict(df_test)
pred2 = pd.DataFrame(pred2)
pred2.columns = ["Death_prediction"]

Sub = pd.read_csv("../input/covid19-global-forecasting-week-1/submission.csv")
Sub.columns
sub_new = Sub[["ForecastId"]]

OP = pd.concat([pred1,pred2,sub_new],axis=1)
OP.head()
OP.columns = ['ConfirmedCases', 'Fatalities', 'ForecastId']
OP = OP[['ForecastId','ConfirmedCases', 'Fatalities']]

OP["ConfirmedCases"] = OP["ConfirmedCases"].astype(int)
OP["Fatalities"] = OP["Fatalities"].astype(int)

OP.head()

You should provide test set for use best model. use_best_model parameter has been switched to false value.


0:	learn: 194.7058770	total: 9.65ms	remaining: 19.3s
1:	learn: 194.6968530	total: 25.7ms	remaining: 25.6s
2:	learn: 194.6836644	total: 209ms	remaining: 2m 19s
3:	learn: 194.6729212	total: 360ms	remaining: 2m 59s
4:	learn: 194.6642225	total: 376ms	remaining: 2m 29s
5:	learn: 194.6549271	total: 392ms	remaining: 2m 10s
6:	learn: 194.6456132	total: 396ms	remaining: 1m 52s
7:	learn: 194.6379830	total: 400ms	remaining: 1m 39s
8:	learn: 194.6254051	total: 576ms	remaining: 2m 7s
9:	learn: 194.6161492	total: 620ms	remaining: 2m 3s
10:	learn: 194.6081001	total: 819ms	remaining: 2m 28s
11:	learn: 194.5953865	total: 836ms	remaining: 2m 18s
12:	learn: 194.5862771	total: 839ms	remaining: 2m 8s
13:	learn: 194.5778946	total: 868ms	remaining: 2m 3s
14:	learn: 194.5676421	total: 1.06s	remaining: 2m 20s
15:	learn: 194.5591361	total: 1.07s	remaining: 2m 13s
16:	learn: 194.5499010	total: 1.26s	remaining: 2m 27s
17:	learn: 194.5404790	total: 1.27s	remaining: 2m 19s
18:	learn: 194.5330344	total: 1.32s	remain

,ForecastId,ConfirmedCases,Fatalities
0,1,424,15
1,2,428,16
2,3,455,17
3,4,456,17
4,5,510,20


In [12]:
OP.to_csv("submission.csv",index=False)